In [ ]:
import os

from aavomics import database

import scrapi

In [ ]:
for alignment in database.ALIGNMENTS:

    for cell_set in database.CELL_SETS:

        # Get the raw Cell Ranger file
        cell_ranger_raw_h5_file_path = cell_set.get_cell_ranger_h5_file_path(filtered=False, alignment_name=alignment.name)

        # Check to make sure the aligned file exists. If not, something's wrong!
        if not os.path.exists(cell_ranger_raw_h5_file_path):
            print("Missing alignment for %s, skipping" % cell_set.name)
            continue

        # Write it out to a file
        anndata_file_path = cell_set.get_anndata_file_path(alignment_name=alignment.name)

        # Check to make sure the converted file doesn't exist
        if os.path.exists(anndata_file_path):
            print("Already converted %s, skipping" % cell_set.name)
            continue

        print("Converting %s" % cell_set.name)
        # Convert the Cell Ranger matrix to anndata
        adata = scrapi.utils.fileio.convert_cellranger_h5_to_anndata(cell_ranger_raw_h5_file_path)

        # Get the filtered matrix also - we will use this to set a flag in the main anndata to track what cells were filtered by Cell Ranger
        cell_ranger_filtered_h5_file_path = cell_set.get_cell_ranger_h5_file_path(filtered=True, alignment_name=alignment.name)
        adata_filtered = scrapi.utils.fileio.convert_cellranger_h5_to_anndata(cell_ranger_filtered_h5_file_path)
        adata.obs["Cell Ranger Called"] = adata.obs.index.isin(adata_filtered.obs.index)
        
        print("Writing %s" % cell_set.name)
        # Write Anndata out to file
        adata.write(anndata_file_path)
        print("Done writing %s" % cell_set.name)